In [ ]:
# Install required packages
!pip install sastrawi tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.34.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


# **Import Library**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, losses, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import random

# **Load Data**

In [ ]:
df=pd.read_csv('dataset_jogja_pre (1).csv')
df

,no,nama,vote_average,vote_count,htm_weekday,htm_weekend,latitude,longitude,type_clean_Agrowisata,type_clean_Alam,...,type_clean_Desa Wisata,type_clean_Kuliner,type_clean_Minat Khusus,type_clean_Museum,type_clean_Pantai,type_clean_Pendidikan,type_clean_Religi,type_clean_Seni,type_clean_Wisata Air,description_clean
0,9,Candi Borobudur,4.7,81922,50000.0,50000.0,-7.607087,110.203623,0,0,...,0,0,0,0,0,0,0,0,0,candi yang pernah masuk sebagai salah satu dar...
1,10,Candi Prambanan,4.7,71751,50000.0,50000.0,-7.751835,110.491532,0,0,...,0,0,0,0,0,0,0,0,0,candi prambanan adalah kompleks candi hindu te...
2,24,Tebing Breksi,4.4,51431,10000.0,10000.0,-7.781477,110.504576,0,1,...,0,0,0,0,0,0,0,0,0,tebing breksi merupakan tempat wisata yang ber...
3,343,Gembira Loka Zoo,4.5,36337,20000.0,25000.0,-7.806234,110.396798,0,0,...,0,0,0,0,0,0,0,0,0,gambira loka adalah kebun binatang yang berada...
4,346,The Palace of Yogyakarta (Keraton Yogyakarta),4.6,30091,8000.0,8000.0,-7.805284,110.364203,0,0,...,0,0,0,0,0,0,0,0,0,kompleks keraton merupakan museum yang menyimp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,139,Pemancingan Adi Winata,5.0,1,0.0,0.0,-7.704577,110.512011,0,0,...,0,0,0,0,0,0,0,0,1,tempat rekreasi keluarga yang menawarkan fasil...
472,105,Ruang Perawatan Jenderal Soedirman,5.0,1,4000.0,4000.0,-7.776474,110.376744,0,0,...,0,0,0,0,0,0,0,0,0,situs sejarah berupa ruangan tempat jenderal s...
473,110,Situs Gedong Pusoko,5.0,1,15000.0,15000.0,-7.807846,110.403758,0,0,...,0,0,0,0,0,0,0,0,0,situs arkeologi yang menyimpan tinggalan sejar...
474,164,Taman Edukasi dan Outbound Sunan Kalijaga,5.0,1,0.0,0.0,-7.809207,110.413252,0,0,...,0,0,0,0,0,0,0,0,0,area edukatif yang menyediakan kegiatan outbou...


# **Preprocessing**

In [ ]:
# Preprocessing teks
descriptions = df['description_clean'].astype(str).tolist()
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(descriptions)
sequences = tokenizer.texts_to_sequences(descriptions)
padded_sequences = pad_sequences(sequences, padding='post', maxlen=100)

In [ ]:
# Buat pasangan data mirip dan tidak mirip
def create_pairs(data):
    pairs = []
    labels = []
    n = len(data)
    for i in range(n):
        # pasangan mirip (label=1)
        j = random.choice([x for x in range(n) if x != i])
        pairs.append([data[i], data[j]])
        labels.append(1 if df.iloc[i]['nama'][:4] == df.iloc[j]['nama'][:4] else 0)  # asumsi mirip jika nama mirip
    return np.array(pairs), np.array(labels)

pairs, labels = create_pairs(padded_sequences)

In [ ]:
# ===============================
# 4. FEATURE ENGINEERING
# ===============================
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
tfidf = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf.fit_transform(df['description_clean'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

type_columns = [col for col in df.columns if col.startswith('type_clean_')]
type_df = df[type_columns]

numeric_features = ['vote_average', 'vote_count', 'htm_weekday', 'htm_weekend']
numeric_df = pd.DataFrame(StandardScaler().fit_transform(df[numeric_features]), columns=numeric_features)

def get_tourism_types(row):
    return [col.replace('type_clean_', '').replace('_', ' ') for col in type_columns if row[col] == 1]

df['tourism_types'] = df.apply(get_tourism_types, axis=1)

all_features = pd.concat([numeric_df, type_df, tfidf_df], axis=1)
feature_dim = all_features.shape[1]

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(pairs, labels, test_size=0.2, random_state=42)

# **Build Model**

In [ ]:
# Model base embedding
def build_base_network(input_shape):
    input = tf.keras.Input(shape=input_shape)
    x = layers.Embedding(input_dim=5000, output_dim=64, input_length=input_shape[0])(input)
    x = layers.Conv1D(64, 5, activation='relu')(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64)(x)
    model = models.Model(input, x)
    return model

In [ ]:
# Jarak cosine sebagai metrik kesamaan
@tf.keras.utils.register_keras_serializable(package="Custom") # Add this decorator
def cosine_distance(vectors):
    x, y = vectors
    x = tf.math.l2_normalize(x, axis=1)
    y = tf.math.l2_normalize(y, axis=1)
    return 1 - tf.reduce_sum(x * y, axis=1, keepdims=True)


In [ ]:
# Jaringan siamese
input_shape = (100,)
base_network = build_base_network(input_shape)

input_a = tf.keras.Input(shape=input_shape)
input_b = tf.keras.Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = layers.Lambda(cosine_distance)([processed_a, processed_b])
model = models.Model([input_a, input_b], distance)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Loss dan compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'mae'])


# Latih model
model.fit([X_train[:, 0], X_train[:, 1]], y_train, batch_size=32, epochs=10, validation_split=0.2)


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 299ms/step - accuracy: 0.9836 - loss: 0.1016 - mae: 0.0543 - val_accuracy: 0.9342 - val_loss: 0.3415 - val_mae: 0.0749
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.9774 - loss: 0.0638 - mae: 0.0277 - val_accuracy: 0.9342 - val_loss: 0.3258 - val_mae: 0.0767
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.9927 - loss: 0.0179 - mae: 0.0124 - val_accuracy: 0.9342 - val_loss: 0.3238 - val_mae: 0.0778
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.9892 - loss: 0.0176 - mae: 0.0136 - val_accuracy: 0.9342 - val_loss: 0.3022 - val_mae: 0.0885
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 1.0000 - loss: 0.0222 - mae: 0.0193 - val_accuracy: 0.9342 - val_loss: 0.2853 - val_mae: 0.1046
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 1.0000 - loss: 0.0086 - mae: 0.0086 - val_accuracy: 0.9342 - val_loss: 0.3062 - val_mae: 0.0980
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms

In [ ]:
# Evaluasi
loss, acc, mae = model.evaluate([X_test[:, 0], X_test[:, 1]], y_test)
print(f"Loss: {loss}, Accuracy: {acc},mae :{mae}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9466 - loss: 0.1624 - mae: 0.0660
Loss: 0.15371105074882507, Accuracy: 0.9479166865348816,mae :0.06736895442008972


In [ ]:
model.save("model.keras")  # Direktori my_model akan berisi SavedModel

# **Berdasarkan Nama Tempat**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Dapatkan embedding semua deskripsi dari model
embedding_model = models.Model(inputs=base_network.input, outputs=base_network.output)
all_embeddings = embedding_model.predict(padded_sequences)

# Fungsi rekomendasi berdasarkan cosine similarity antar embedding
def get_recommendations(place_name, top_n=10):
    if place_name not in df['nama'].values:
        print("Nama tempat tidak ditemukan.")
        return None

    idx = df[df['nama'] == place_name].index[0]
    place_embedding = all_embeddings[idx].reshape(1, -1)

    similarities = cosine_similarity(place_embedding, all_embeddings)[0]
    sim_indices = similarities.argsort()[::-1][1:top_n+1]

    # Ambil semua kolom yang merupakan tipe
    type_columns = [col for col in df.columns if col.startswith('type_clean_')]

    # Ambil kolom yang ingin ditampilkan
    hasil = df.iloc[sim_indices][['nama', 'description_clean', 'htm_weekday', 'htm_weekend'] + type_columns].copy()
    hasil['similarity'] = similarities[sim_indices]

    # Buat kolom type sebagai list dari type_columns yang bernilai 1
    def ambil_tipe(row):
        return [col.replace('type_clean_', '').replace('_', ' ') for col in type_columns if row[col] == 1]

    hasil['type'] = hasil.apply(ambil_tipe, axis=1)
    hasil = hasil[['nama', 'description_clean', 'type', 'htm_weekday', 'htm_weekend', 'similarity']]

    return hasil.reset_index(drop=True)


# Contoh penggunaan
print(get_recommendations('Malioboro'))


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
                                 nama  \
0                                Tugu   
1     Museum Batik Keraton Yogyakarta   
2  Dewi Kano Tourism Village Kaligono   
3             Brug Londho River Track   
4                        Teras Merapi   
5                     Tugu Mintoragan   
6                 Desa Wisata Trumpon   
7        Gunung Api Purba Nglanggeran   
8                         Gua Permoni   
9      TAMAN GROUNDSILL BENDUNG TEGAL   

                                   description_clean                  type  \
0  tugu ini sekarang merupakan salah satu objek p...  [Budaya Dan Sejarah]   
1  di museum batik keraton yogyakarta wisatawan a...  [Budaya Dan Sejarah]   
2  desa wisata yang berfokus pada konservasi alam...          [Agrowisata]   
3  jalur wisata alam di ngaglik sleman yang menaw...          [Agrowisata]   
4  destinasi dengan pemandangan langsung ke gunun...                [Alam]   
5  monumen penghargaan dengan nilai seja

# Konversi

In [ ]:
model.save('model_wisata.h5')

In [ ]:
!pip install tensorflowjs

In [ ]:
import tensorflow as tf

# Load the model with the custom object
model1 = tf.keras.models.load_model('model_wisata.h5', custom_objects={'cosine_distance': cosine_distance})

tf.saved_model.save(model1, 'modelwisata_saved_model')

converter = tf.lite.TFLiteConverter.from_saved_model('modelwisata_saved_model')
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  --output_format=tfjs_graph_model \
  modelwisata_saved_model \
  model_tfjs/

2025-06-04 16:33:30.616980: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749054810.631536     761 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749054810.635791     761 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
2025-06-04 16:33:41.148631: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1749054821.494149     761 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00

In [ ]:
import shutil

shutil.make_archive('model_tfjs', 'zip', 'model_tfjs')


'/content/model_tfjs.zip'